<a href="https://colab.research.google.com/github/EnricoHuber/TensorFlow-Review/blob/main/Classification_with_TF_and_IRIS_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import pandas as pd

# Get the Data

In [ ]:
columns = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
species = ['Setosa', 'Versicolor', 'Virginica']

In [ ]:
# This cell get the path of file, next we will be needing to get the data from the file
train_path = tf.keras.utils.get_file('iris_training.csv', 'https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv')
test_path = tf.keras.utils.get_file('iris_training.csv', 'https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv')

In [ ]:
train = pd.read_csv(train_path, names=columns, header=0)
test = pd.read_csv(test_path, names=columns, header=0)

In [ ]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [ ]:
y_train = train.pop('Species')
y_test = test.pop('Species')

In [ ]:
train.shape, test.shape

((120, 4), (120, 4))

In [ ]:
def input_fn(features, labels, training=True, batch_size=256):
  dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

  if training:
    dataset = dataset.shuffle(1000).repeat()
  return dataset.batch(batch_size)

In [ ]:
feature_columns = []
for key in train:
  feature_columns.append(tf.feature_column.numeric_column(key))

In [ ]:
feature_columns

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

# Build the model
We are going to use pre-built TF classification model:



*   DNNClassifier (Deep Neural Network)
*   LinearClassifier



In [ ]:
classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[30, 10],
    n_classes=3
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp4kpkaob9', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [47]:
classifier.train(
    input_fn = lambda: input_fn(train, y_train, training=True),
    steps=5000
)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp4kpkaob9/model.ckpt-5000
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 5000...
INFO:tensorflow:Saving checkpoints for 5000 into /tmp/tmp4kpkaob9/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 5000...
INFO:tensorflow:loss = 0.30324942, step = 5000
INFO:tensorflow:global_step/sec: 471.476
INFO:tensorflow:loss = 0.31887805, step = 5100 (0.214 sec)
INFO:tensorflow:global_step/sec: 614.346
INFO:tensorflow:loss = 0.29081744, step = 5200 (0.165 sec)
INFO:tensorflow:global_step/sec: 597.754
INFO:tensorflow:loss = 0.297664, step = 5300 (0.167 sec)
INFO:tensorflow:global_step/sec: 576.519

In [48]:
results = classifier.evaluate(
    input_fn = lambda: input_fn(test, y_test, training=False)
)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-05-04T09:17:18Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp4kpkaob9/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.20534s
INFO:tensorflow:Finished evaluation at 2021-05-04-09:17:18
INFO:tensorflow:Saving dict for global step 10000: accuracy = 0.975, average_loss = 0.20238523, global_step = 10000, loss = 0.20238523
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 10000: /tmp/tmp4kpkaob9/model.ckpt-10000


In [49]:
print('Test Set Accuracy: ', {results['accuracy']})

Test Set Accuracy:  {0.975}


In [89]:
def input_fn(features, batch_size=256):
  return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)


features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
predict = {}

print('Type Numeric Values')
for feature in features:
  valid = True
  while valid:
    val = input(feature + ': ')
    if not val.isdigit():
      valid = False
  predict[feature] = [float(val)]


predictions = classifier.predict(
    input_fn = lambda: input_fn(predict)
    )

Type Numeric Values
SepalLength: 0.1
SepalWidth: 0.3
PetalLength: 0.5
PetalWidth: 0.4


In [82]:
predictions

<generator object Estimator.predict at 0x7f8e2bc41b50>

In [94]:
for pred_dict in predictions:
  print(pred_dict)
  class_id = pred_dict['class_ids'][0]
  probability = pred_dict['probabilities'][class_id]

In [93]:
print(f'Flower is a {species[class_id]} with a probability of {round(probability*100, 2)}%')

Flower is a Versicolor with a probability of 39.76%
